We create a pickle file fom csv files
- dict of dataframes

In [1]:
import pandas as pd
import os
import numpy as np
import copy
from voxcell import RegionMap
import pickle


In [2]:
import sys
sys.path.append('/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/notebooks/scripts/')

from helper_functions import get_all_filenames, get_csv_filenames, extract_prefix_from_filenames
from helper_functions import extract_regions_from_column_names, read_and_concat_csv_files_new, combine_rows_and_calculate_average

In [3]:
#Get all substructure volume data
file =  '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/parcellation_to_parcellation_term_membership_extend.csv'
parcellation_annotation = pd.read_csv(file)
print("number of memberships at all levels:",len(parcellation_annotation))
parcellation_annotation = parcellation_annotation[parcellation_annotation['parcellation_term_set_name'] == 'substructure'] 
print("number of memberships at substructure level:",len(parcellation_annotation))
volumes = parcellation_annotation[['parcellation_label', 'parcellation_index', 'parcellation_term_acronym', 'parcellation_term_set_name', 'parcellation_term_name', 'voxel_count', 'volume_mm3']]
substructure_vol = volumes[volumes['parcellation_term_set_name'] == 'substructure']

number of memberships at all levels: 3489
number of memberships at substructure level: 737


I have a df column volumes['parcellation_term_acronym'] where I want to remove all / from every element if it's part of the string. 

In [4]:
folder = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/'

In [5]:
from helper_functions import get_nrrd_files

save_nrrd = f'{folder}nrrd/'
filenames = get_all_filenames(folder)

created_nrrds, nrrd_files_without_extension = get_nrrd_files(save_nrrd)

In [6]:
import re
# Pattern to match the numeric part of filenames
pattern = re.compile(r'(\d{4})')  # Matches four consecutive digits

# Set to store existing numbers
existing_numbers = set()

for filename in nrrd_files_without_extension:
    # Match numeric part of filename
    match = pattern.search(filename)
    if match:
        existing_numbers.add(int(match.group(1)))

# Find missing numbers between 1 and 5300
missing_numbers = sorted(set(range(1, 5323)) - existing_numbers)

In [7]:
#Check the number of cell types which do not have an nrrd file
len(missing_numbers)

5322

### Continuation

In [8]:
%%time

# Get all regional density data
folder_path = f'{folder}csv/'
filenames = get_all_filenames(folder_path)
csv_filenames = get_csv_filenames(folder_path)
prefixes = extract_prefix_from_filenames(csv_filenames)
unique_prefixes = sorted(list(set(prefixes)))
#The next line is different because unique_prefixes is different than how Allen named cell types
allen_regions = extract_regions_from_column_names(folder_path, csv_filenames)
result_dataframes = read_and_concat_csv_files_new(csv_filenames, unique_prefixes, folder_path)

CPU times: user 1.86 s, sys: 111 ms, total: 1.98 s
Wall time: 2 s


In [9]:
# Update result_dataframes with the real cell type names
result_dataframes = {allen_regions[i]: df for i, (_, df) in enumerate(result_dataframes.items())}

In [10]:
combined_result_dataframes = combine_rows_and_calculate_average(result_dataframes)

- I have a dict of dfs (result_dataframes) where result_dataframes.keys() contains all region names. Each df contains a column of density numbers e.g. result_dataframes['AAA']['concatenated_density']
- There are no duplicate cell-types in the dfs
- I need to calculate volume from appropriate annotation volume voxels (what was used in step 2 script)


We will continue with calculating the total cells in the brain using cell-type densities and region volumes

In [11]:
import nrrd, json
#Read CCFv3 annotation volumes (choose 1)
# Specify the full or relative path to the log file
# #1
# data_folder = "/gpfs/bbp.cscs.ch/project/proj84/piluso/share/general/warped_augmented_CCFv3/"
# CCFv3_0, _ = nrrd.read(f'{data_folder}annotation_25_2022_CCFv3_0.nrrd')
# #2
# data_folder = "/gpfs/bbp.cscs.ch/data/project/proj62/csaba/atlas/bbp_prod_files/2022/"
# CCFv3_0, _ = nrrd.read(f'{data_folder}annotation_25.nrrd')
# #3
data_folder = "/gpfs/bbp.cscs.ch/project/proj84/piluso/share/general/warped_augmented_CCFv3/"
CCFv3_0, _ = nrrd.read(f'{data_folder}annotation_25_2022_CCFv3a.nrrd')
# CCFv3_0, _ = nrrd.read("/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/annotation_ccfv3_l23split_barrelsplit_validated.nrrd")

#region_map_path ='/gpfs/bbp.cscs.ch/project/proj62/csaba/atlas/bbp_prod_files/1.json'
region_map_path = '/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/hierarchy_ccfv3_l23split_barrelsplit.json'
region_map = RegionMap.load_json(region_map_path)


In [12]:
# Edge length of the voxel in millimeters
edge_length_mm = 25.0 / 1000.0  # Convert micrometers to millimeters

# Calculate the volume of the voxel in cubic millimeters
volume25_mm3 = round(edge_length_mm ** 3, 10)

print("Volume of the voxel in mm^3:", volume25_mm3)

# Edge length of the voxel in millimeters
edge_length_mm = 10.0 / 1000.0  # Convert micrometers to millimeters

# Calculate the volume of the voxel in cubic millimeters
volume10_mm3 = round(edge_length_mm ** 3, 10)

print("Volume of the voxel in mm^3:", volume10_mm3)


Volume of the voxel in mm^3: 1.5625e-05
Volume of the voxel in mm^3: 1e-06


From density we calculate total number of cells: density * volume

In [13]:
%%time 
total_cells = {}

if CCFv3_0.shape == (528, 320, 456):
    print("annotation vol is 25 um resolution")
    for region_name, df in combined_result_dataframes.items():
        # Get the corresponding volume for the region
        label = volumes.loc[volumes['parcellation_term_acronym'] == region_name, 'parcellation_label'].values[0]
        id_ = int(label.rsplit('-', 1)[-1])
        #id_ = region_map.find(region_name, attr='name', ignore_case=True, with_descendants=False)
        volume = np.count_nonzero(CCFv3_0 == id_) * volume25_mm3
        #print(region_name, label, id_, f"{volume:.6f}")
        # Calculate total cells by multiplying density by volume
        total_cells[region_name] = pd.DataFrame({'total_cellnumber': df['density_mm3'] * volume})

elif CCFv3_0.shape == (566, 320, 456):
    print("annotation vol is 25 um resolution but it is larger than the vanilla CCFv3")
    for region_name, df in combined_result_dataframes.items():
        # Get the corresponding volume for the region
        label = volumes.loc[volumes['parcellation_term_acronym'] == region_name, 'parcellation_label'].values[0]
        id_ = int(label.rsplit('-', 1)[-1])
        #id_ = region_map.find(region_name, attr='name', ignore_case=True, with_descendants=False)
        volume = np.count_nonzero(CCFv3_0 == id_) * volume25_mm3
        #print(region_name, label, id_, f"{volume:.6f}")
        # Calculate total cells by multiplying density by volume
        total_cells[region_name] = pd.DataFrame({'total_cellnumber': df['density_mm3'] * volume})

elif CCFv3_0.shape == (1320, 800, 1140) or CCFv3_0.shape == (1140, 800, 1320):
    print("*** annotation vol is 10 um resolution *** ")
    for region_name, df in combined_result_dataframes.items():
        # Get the corresponding volume for the region
        label = volumes.loc[volumes['parcellation_term_acronym'] == region_name, 'parcellation_label'].values[0]
        id_ = int(label.rsplit('-', 1)[-1])
        #id_ = region_map.find(region_name, attr='name', ignore_case=True, with_descendants=False)
        #print(region_name, label, id_)
        
        if CCFv3_0.shape == (1320, 800, 1140):
            volume = np.count_nonzero(CCFv3_0 == id_) * volume10_mm3
        elif CCFv3_0.shape == (1140, 800, 1320):
            volume = np.count_nonzero(CCFv3_0 == id_) * volume10_mm3 
        # Calculate total cells by multiplying density by volume
        print(region_name, label, id_, f"{volume:.6f}")
        total_cells[region_name] = pd.DataFrame({'total_cellnumber': df['density_mm3'] * volume})
else:
    print("*** Annotation volume shape is not understood. *** ")

annotation vol is 25 um resolution but it is larger than the vanilla CCFv3
CPU times: user 50.1 s, sys: 10.5 s, total: 1min
Wall time: 1min


In [14]:
# Assuming 'total_cells' is your dictionary of DataFrames
with open(f'{folder}total_cells.pickle', 'wb') as f:
    pickle.dump(total_cells, f)

print(f"Saved here: {folder}")

Saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/


In [15]:
# Create a new dictionary to store the sums for each region
collapsed_data = {region: df['total_cellnumber'].sum() for region, df in total_cells.items()}

# Create a new DataFrame from the collapsed_data dictionary
collapsed_df = pd.DataFrame.from_dict(collapsed_data, orient='index', columns=['sum_concatenated_density'])

# Reset the index to make the region names a column instead of the index
collapsed_df.reset_index(inplace=True)

# Rename the columns to match the desired output
collapsed_df.columns = ['region_name', 'SUM_of_Cells']

# Convert 'sum_concatenated_density' column values to integers
collapsed_df['SUM_of_Cells'] = collapsed_df['SUM_of_Cells'].astype(int)


In [16]:
# Print the total sum of cells in a human-readable format
print("Total sum of cells:", '{:,.0f}'.format(collapsed_df['SUM_of_Cells'].sum()))

Total sum of cells: 93,876,733


## Neuron / non-neuron ratio

In [17]:
meta_path = "/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/WMB-10X/20231215/views/cell_metadata_with_cluster_annotation.csv"
metadata = pd.read_csv(meta_path,  usecols=['class', 'cluster'], dtype={'cell_label':str})


In [18]:
n_classes = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
       '04 DG-IMN Glut', '05 OB-IMN GABA', '06 CTX-CGE GABA',
       '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
       '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '13 CNU-HYa Glut',
       '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
       '18 TH Glut', '19 MB Glut', '20 MB GABA', '21 MB Dopa',
       '22 MB-HB Sero', '23 P Glut', '24 MY Glut', '25 Pineal Glut',
       '26 P GABA', '27 MY GABA', '28 CB GABA', '29 CB Glut',]

nn_classes = ['30 Astro-Epen', '31 OPC-Oligo', '32 OEC', '33 Vascular',
       '34 Immune']

exc = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
      '04 DG-IMN Glut', '13 CNU-HYa Glut', '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
      '18 TH Glut', '19 MB Glut', '23 P Glut', '24 MY Glut', '25 Pineal Glut', '29 CB Glut',]
inh = ['05 OB-IMN GABA', '06 CTX-CGE GABA', '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
      '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '20 MB GABA', '26 P GABA', '27 MY GABA', '28 CB GABA', ]
other = ['21 MB Dopa', '22 MB-HB Sero', ]

neurontypes = np.unique(metadata[metadata['class'].isin(n_classes)]['cluster'].values)
nonneurontypes = np.unique(metadata[metadata['class'].isin(nn_classes)]['cluster'].values)
exctypes = np.unique(metadata[metadata['class'].isin(exc)]['cluster'].values)
inhtypes = np.unique(metadata[metadata['class'].isin(inh)]['cluster'].values)
othertypes = np.unique(metadata[metadata['class'].isin(other)]['cluster'].values)

In [19]:
total_cells_copy = copy.deepcopy(total_cells)

In [20]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(nonneurontypes)]
    
# Create a new dictionary to store the sums for each region
collapsed_data = {region: df['total_cellnumber'].sum() for region, df in total_cells_copy.items()}

# Create a new DataFrame from the collapsed_data dictionary
collapsed_df = pd.DataFrame.from_dict(collapsed_data, orient='index', columns=['sum_concatenated_density'])

# Reset the index to make the region names a column instead of the index
collapsed_df.reset_index(inplace=True)

# Rename the columns to match the desired output
collapsed_df.columns = ['region_name', 'SUM_of_Cells']

# Convert 'sum_concatenated_density' column values to integers
collapsed_df['SUM_of_Cells'] = collapsed_df['SUM_of_Cells'].astype(int)

# Print the total sum of cells in a human-readable format
print("Total sum of non-neurons:", '{:,.0f}'.format(collapsed_df['SUM_of_Cells'].sum()))

Total sum of non-neurons: 41,341,793


In [21]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(neurontypes)]
    
# Create a new dictionary to store the sums for each region
collapsed_data = {region: df['total_cellnumber'].sum() for region, df in total_cells_copy.items()}

# Create a new DataFrame from the collapsed_data dictionary
collapsed_df = pd.DataFrame.from_dict(collapsed_data, orient='index', columns=['sum_concatenated_density'])

# Reset the index to make the region names a column instead of the index
collapsed_df.reset_index(inplace=True)

# Rename the columns to match the desired output
collapsed_df.columns = ['region_name', 'SUM_of_Cells']

# Convert 'sum_concatenated_density' column values to integers
collapsed_df['SUM_of_Cells'] = collapsed_df['SUM_of_Cells'].astype(int)

# Print the total sum of cells in a human-readable format
print("Total sum of neurons:", '{:,.0f}'.format(collapsed_df['SUM_of_Cells'].sum()))

Total sum of neurons: 52,534,615


In [22]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(exctypes)]
    
# Create a new dictionary to store the sums for each region
collapsed_data = {region: df['total_cellnumber'].sum() for region, df in total_cells_copy.items()}

# Create a new DataFrame from the collapsed_data dictionary
collapsed_df = pd.DataFrame.from_dict(collapsed_data, orient='index', columns=['sum_concatenated_density'])

# Reset the index to make the region names a column instead of the index
collapsed_df.reset_index(inplace=True)

# Rename the columns to match the desired output
collapsed_df.columns = ['region_name', 'SUM_of_Cells']

# Convert 'sum_concatenated_density' column values to integers
collapsed_df['SUM_of_Cells'] = collapsed_df['SUM_of_Cells'].astype(int)

# Print the total sum of cells in a human-readable format
print("Total sum of excitatory neurons:", '{:,.0f}'.format(collapsed_df['SUM_of_Cells'].sum()))

Total sum of excitatory neurons: 30,757,245


In [23]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(inhtypes)]
    
# Create a new dictionary to store the sums for each region
collapsed_data = {region: df['total_cellnumber'].sum() for region, df in total_cells_copy.items()}

# Create a new DataFrame from the collapsed_data dictionary
collapsed_df = pd.DataFrame.from_dict(collapsed_data, orient='index', columns=['sum_concatenated_density'])

# Reset the index to make the region names a column instead of the index
collapsed_df.reset_index(inplace=True)

# Rename the columns to match the desired output
collapsed_df.columns = ['region_name', 'SUM_of_Cells']

# Convert 'sum_concatenated_density' column values to integers
collapsed_df['SUM_of_Cells'] = collapsed_df['SUM_of_Cells'].astype(int)

# Print the total sum of cells in a human-readable format
print("Total sum of inhibitory neurons:", '{:,.0f}'.format(collapsed_df['SUM_of_Cells'].sum()))

Total sum of inhibitory neurons: 21,605,821


In [24]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(othertypes)]
    
# Create a new dictionary to store the sums for each region
collapsed_data = {region: df['total_cellnumber'].sum() for region, df in total_cells_copy.items()}

# Create a new DataFrame from the collapsed_data dictionary
collapsed_df = pd.DataFrame.from_dict(collapsed_data, orient='index', columns=['sum_concatenated_density'])

# Reset the index to make the region names a column instead of the index
collapsed_df.reset_index(inplace=True)

# Rename the columns to match the desired output
collapsed_df.columns = ['region_name', 'SUM_of_Cells']

# Convert 'sum_concatenated_density' column values to integers
collapsed_df['SUM_of_Cells'] = collapsed_df['SUM_of_Cells'].astype(int)

# Print the total sum of cells in a human-readable format
print("Total sum of modulatory neurons:", '{:,.0f}'.format(collapsed_df['SUM_of_Cells'].sum()))

Total sum of modulatory neurons: 171,163
